## TO: Visualization of examples

In [14]:
#Import requirements download
from bs4 import BeautifulSoup
import urllib
import wget
import re
from robobrowser import RoboBrowser
import pandas as pd
import warnings
import datetime
import astropy.time as astime
warnings.filterwarnings("ignore")
import zipfile  
#import requirements plot
from astropy.io import fits
from astropy import wcs
import sys
import os
import glob
import cv2
import gzip
import numpy as np
import pandas as pd
# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib
#matplotlib.rc_file("../../templates/matplotlibrc")
#import matplotlib.py

## 1. Create DF with some instances

In [15]:
list_CRTS_id = ['CSS160605:113003+280648', 'CSS160528:162350-121731', 'CSS151204:035824+395528', 
                'CSS160910:164807+543702', 'CSS160530:100805+465751', 'CSS101017:014532+253215',
                'CSS151102:011316-063052', 'CSS130509:123251+123111', 'CSS120330:121724-135621',
                'CSS170519:155520+421151', 
                'CSS160704:223451+084838', 'CSS151216:044922+183333', 'CSS101211:112729-043718',
                'CSS160926:214928-090031', 'CSS160112:043229+311222'
               ]
list_RA = ['172.51287', '245.96030', '59.59954', 
           '252.02731', '152.02003', '26.38296', 
           '18.31757', '188.21436', '184.34835',
           '238.83319', 
           '338.71418', '72.34222', '171.87261',
           '327.36497', '68.11962'       
          ]
list_DEC = ['28.11336', '-12.29200', '39.92433', 
            '54.61719', '46.96420', '25.53758', 
            '-6.51454', '12.51982', '-13.93905',
            '42.19743', 
            '8.81058', '18.55922', '-4.62174',
            '-9.00869', '31.20620'     
           ]
list_mag = [18.58, 17.24, 17.77,
            18.52, 19.19, 17.53,
            18.55, 18.54, 19.19,
            17.97, 
            19.67, 18.11, 19.57, 
            19.29, 17.49
            
           ]
list_mag_err = [3.22, 2.06, 3.23, 
                2.48, 2.61, -1.53,
                0, 0, 0,
                0,  
                0, 0, 0,
                0, 0  
               ]
list_object_type = ['CV', 'CV', 'CV',
                   'SN', 'SN', 'SN',
                   'BZ', 'BZ', 'BZ',
                    'FLARES',
                    'AGN', 'AGN', 'AGN',
                    'OTHER', 'OTHER'
                     ]
list_url_LC = ['http://nesssi.cacr.caltech.edu/catalina/20160605/1606051260564134509p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160528/1605280120874164002p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20151204/1512041400174114620p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160910/1609101550544114729p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160530/1605301460394125395p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20101017/1010171260094102098p.html', 
               'http://nesssi.cacr.caltech.edu/catalina/20151102/1511020070074126189p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20130509/1305091120664119543p.html', 
               'http://nesssi.cacr.caltech.edu/catalina/20120330/1203300120654103092p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20170519/1705191410434147580p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160704/1607041091204106148p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20151216/1512161180254136858p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20101211/1012110040624115724p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160926/1609260091164137445p.html',
               'http://nesssi.cacr.caltech.edu/catalina/20160112/1601121320214105229p.html'               
              ]

In [16]:
#create DF with selected objects
data = {'CRTS ID' : list_CRTS_id, 'RA' : list_RA, 
        'DEC' : list_DEC, 'Mag': list_mag, 
       'Class' : list_object_type, 'URL LC' : list_url_LC}
df_objects =  pd.DataFrame(data)
df_objects.head(15)

,CRTS ID,Class,DEC,Mag,RA,URL LC
0,CSS160605:113003+280648,CV,28.11336,18.58,172.51287,http://nesssi.cacr.caltech.edu/catalina/201606...
1,CSS160528:162350-121731,CV,-12.29200,17.24,245.96030,http://nesssi.cacr.caltech.edu/catalina/201605...
2,CSS151204:035824+395528,CV,39.92433,17.77,59.59954,http://nesssi.cacr.caltech.edu/catalina/201512...
3,CSS160910:164807+543702,SN,54.61719,18.52,252.02731,http://nesssi.cacr.caltech.edu/catalina/201609...
4,CSS160530:100805+465751,SN,46.96420,19.19,152.02003,http://nesssi.cacr.caltech.edu/catalina/201605...
5,CSS101017:014532+253215,SN,25.53758,17.53,26.38296,http://nesssi.cacr.caltech.edu/catalina/201010...
6,CSS151102:011316-063052,BZ,-6.51454,18.55,18.31757,http://nesssi.cacr.caltech.edu/catalina/201511...
7,CSS130509:123251+123111,BZ,12.51982,18.54,188.21436,http://nesssi.cacr.caltech.edu/catalina/201305...
8,CSS120330:121724-135621,BZ,-13.93905,19.19,184.34835,http://nesssi.cacr.caltech.edu/catalina/201203...
9,CSS170519:155520+421151,FLARES,42.19743,17.97,238.83319,http://nesssi.cacr.caltech.edu/catalina/201705...


In [20]:
df_objects.to_pickle('/home/oem/Documentos/Astronomy/CRTS/Data' + '/' + 'TOsample.pkl')

## Function to download LCs

In [17]:
#function that gets the LC points given an url
#url_LC is a string
def get_LC_points(url_LC):
    aux_LC = urllib.request.urlopen(url_LC).read()
    soup_LC = BeautifulSoup(aux_LC, "lxml")
    #each point has a tag <area
    area_rows = soup_LC.find_all("area")
    LC_x_list = []
    LC_y_list = []
    for aux in area_rows:
        #onmouse over is the key that has the (x,y) points
        list_str_points = aux['onmouseover'].split()
        #position 0 has the x value
        complete_x = list_str_points[0]
        x_value = complete_x[complete_x.find("('")+2:]
        #change from mjd 
        x_year = astime.Time(float(x_value) + 53249, format='mjd')
        x_decimal_year = x_year.decimalyear
        LC_x_list.append(x_decimal_year)
        #position 2 has the y value
        complete_y = list_str_points[2]
        y_value = complete_y[complete_y.find("('")+2:complete_y.find("')")]
        LC_y_list.append(float(y_value))
    LC_array = np.array([LC_x_list, LC_y_list])
    LC_array = LC_array.T
    return LC_array

## Function to download images

In [18]:
#do this for each TO
import os
def makedir(outdir):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
        
CRTS_DR3_beta_url = 'http://crts.iucaa.in/crts/'
browser = RoboBrowser()
browser.open(CRTS_DR3_beta_url)
form = browser.get_form(action='/crts/imaging')
# print(form)
def download_images_per_object (crds_RA, crds_DEC, obj_class, object_id):
    #crdsRA/DEC must be a string 
    #form['imaging-search-query'].value = '332.64277 -22.66323' #'X' + ' ' + 'Y'
    form['imaging-search-query'].value = crds_RA + ' ' + crds_DEC
    form['imaging-search-radius'].value = '0.1'
    browser.submit_form(form) 
    #search: se actualiza la variable de browser 
    #duda: hay una forma de pedir la nueva url de esa pagina?

    #Find all the "values" or images for that object
    num = browser.find("div", {"class" : "Found"})
    complete_string = num.getText()
    total = [int(s) for s in complete_string.split() if s.isdigit()] #extraer el numero que corresponde a las ims
    list_of_numbers = list(range(0, int(total[0])-1))
    str_list_values = []
    for i in range(int(total[0])):
        a = i
        str_list_values.append(str(a))
    
    #extract information from the table
    #browser_html_code = browser.parsed() #type: <class 'bs4.element.ResultSet'>
    #first_element = browser_html_code[0] #type: <class 'bs4.element.Tag'>
    #text = first_element.getText()

    #x = browser.find("div", {"id" : "outTable"})
    #select only the table 
    read_table = browser.find("table", {"class" : "sortable"})  #type: <class 'bs4.element.Tag'>
    string_all_rows_table = read_table.getText()
    #print('what I found:', string_all_rows_table)
    list_all_rows_table = string_all_rows_table.strip().split()
    data_information = list_all_rows_table[4:]
    #extract info from each column
    center_ra_list = data_information[::4]
    center_dec_list = data_information[1::4] 
    obs_date_list = data_information[2::4]
    file_name_list = data_information[3::4]
    #create dataframe
    data = {list_all_rows_table[0] : center_ra_list, list_all_rows_table[1] : center_dec_list, 
        list_all_rows_table[2] : obs_date_list, list_all_rows_table[3] : file_name_list}
    df_per_object = pd.DataFrame(data)
    df_per_object = df_per_object.sort_values('OBS-DATE')
    df_per_object.head()
    #df_per_object.to_pickle(complete_path + file_name[0:-4] + '.pkl')
    #print('html code:', browser.parsed()) #lo que está viendo de html

    #Proceed to download all the images of the object 
    print('*************')
    new_form = browser.get_form(action='/crts/download')
    print(new_form)
    # image = browser.get_form(class_='imaging-search-query')
    #print(image)
    # extraer todos los values con browser.find

    #new_form['downloadSelector'] = ['49','2'] #chulear el que tenga ese value
    new_form['downloadSelector'] = str_list_values
    browser.submit_form(new_form) #queda con un click here to download/se actualiza la pagina

    #se abre otra pagina
    link =browser.find("a", { "class" : "downloadLink" })
    download_zip = link.get('href')
    base_url = 'http://crts.iucaa.in'
    download_zip = base_url + download_zip
    data_dir = '/home/oem/Documentos/Astronomy/CRTS/Data/'
    #object_class = obj_class
    #file_name = 'first_object.zip'
    file_name = object_id 
    complete_path = data_dir + obj_class + '/' + file_name + '.zip'
    wget.download(download_zip, complete_path)  
    makedir(complete_path[0:-4])
    zip = zipfile.ZipFile(complete_path)  
    zip.extractall(complete_path[0:-4])  
    #print(download_zip)
    #save DF file with metadata
    df_per_object.to_pickle(complete_path[0:-4] + '/' + file_name + '.pkl')
    return df_per_object


In [19]:
#download images and LC for each object
path_to_save_LC = '/home/oem/Documentos/Astronomy/CRTS/Data/'
for index, row in df_objects.iterrows():
    file_name_from_df = row['CRTS ID']
    ra_crd_from_df = row['RA']
    dec_crd_from_df = row['DEC']
    class_from_Df = row['Class']
    url_from_df = row['URL LC']
    df_aux = download_images_per_object(ra_crd_from_df, dec_crd_from_df, class_from_Df, file_name_from_df)
    url_array = get_LC_points(url_from_df)
    np.save(path_to_save_LC + class_from_Df + '/' + file_name_from_df + '/' + 'LC_' + file_name_from_df+'.npy', 
            url_array)


*************
<RoboForm resultsTable=/opt/www/crtsDR3/crtsDR3/staging/im172p51287_28p11336_0p1_010029476259.dat, ra=172.51287, dec=28.11336, radius=0.1, imageType=CutOuts, downloadSelector=[]>
*************
<RoboForm resultsTable=/opt/www/crtsDR3/crtsDR3/staging/im245p9603_-12p292_0p1_010106234970.dat, ra=245.9603, dec=-12.292, radius=0.1, imageType=CutOuts, downloadSelector=[]>
*************
<RoboForm resultsTable=/opt/www/crtsDR3/crtsDR3/staging/im59p59954_39p92433_0p1_010142889678.dat, ra=59.59954, dec=39.92433, radius=0.1, imageType=CutOuts, downloadSelector=[]>
*************
<RoboForm resultsTable=/opt/www/crtsDR3/crtsDR3/staging/im252p02731_54p61719_0p1_010208712287.dat, ra=252.02731, dec=54.61719, radius=0.1, imageType=CutOuts, downloadSelector=[]>
*************
<RoboForm resultsTable=/opt/www/crtsDR3/crtsDR3/staging/im152p02003_46p9642_0p1_010259049304.dat, ra=152.02003, dec=46.9642, radius=0.1, imageType=CutOuts, downloadSelector=[]>
*************
<RoboForm resultsTable=/opt/w